In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
sec = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=sec

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

f:\Programming 2\GENAI(GitHub)\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone.vectorstores import PineconeVectorStore
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get("PINECONE_API_KEY")
environment = os.environ.get("PINECONE_ENVIRONMENT")
pc = Pinecone(api_key=api_key)

# Ensure the index exists
index_name = "chatinit"  # Replace with your actual index name
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
index = pc.Index(index_name)
vectore_Store = PineconeVectorStore(index=index, embedding=embedding_model)

In [41]:
retriever = vectore_Store.as_retriever(search_type="similarity", search_kwargs={
        "namespace": "solvrz", "k": 3
    })

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [31]:
custom_prompt = PromptTemplate(
    template="""
You are a bot with the following enabled services: {enabled_services}.
Your task is to analyze the user's query and determine whether it falls under the category of "sales," "support," or is a "general" query.
Based on the category, respond accordingly, considering the context of the ongoing conversation.

**Context:**
{context}

**Chat History:**
{chat_history}

**User Query:**
{input}

**Instructions:**
1. **Categorization:**
   - Analyze the query and classify it as "sales," "support," or "general."
2. **Response Based on Category:**
   - If the query relates to sales, provide a sales-specific answer.
   - If it relates to support, offer guidance or troubleshooting steps.
   - For general queries, give helpful insights.
3. **Tone and Style:**
   - Use a professional and friendly tone.
   - Keep responses concise and contextually relevant.
""",
    input_variables=["enabled_services", "chat_history", "input", "context"],
)


In [42]:
combine_docs_chain = create_stuff_documents_chain(llm, custom_prompt)

# Step 3: Create the retrieval chain using your custom chain
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [65]:
enabled_services = "sales, support, feedback, appointment booking, learn more"
query = "How can I contact with solvrz?"
chat_history = []
context = "Relevant documents retrieved based on the user's query."

In [66]:
response = rag_chain.invoke({
    "enabled_services": enabled_services,
    "chat_history": chat_history,
    "input": query,
    "context": context
})


In [67]:
answer = response.get("answer", "No answer found")
print(answer)

**1. Categorization:** General

**2. Response:**

We'd love to connect with you!  You can reach Solvrz in a few ways:

* **For sales inquiries:** Please email us at [sales@solvrz.com - replace with actual email]
* **For support related to our products (e.g., Chatinit, dCopyright):**  Please email us at [support@solvrz.com - replace with actual email]
* **For general inquiries or partnership opportunities:** Please email us at [info@solvrz.com - replace with actual email]

We look forward to hearing from you and exploring how we can help you unlock possibilities through innovative software solutions.

